# 3. Generating Multiple Samples using MS1 Controller

In this notebook, we demonstrate how ViMMS can be used to generate multiple samples (sets of chemicals) that are biological and technical replicates. The MS1 controller is then used to produce mass spectral data in form of .mzML files for the multiple samples.

In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
from collections import defaultdict
import os
from pathlib import Path

In [4]:
import sys
sys.path.append('..')

In [5]:
from vimms.Chemicals import ChemicalCreator, MultiSampleCreator
from vimms.MassSpec import IndependentMassSpectrometer
from vimms.Controller import SimpleMs1Controller
from vimms.Common import *

Load previously trained KDEs in `PeakSampler` and the list of extracted metabolites, created in **01. Download Data.ipynb**.

In [6]:
base_dir = os.path.abspath('example_data')
ps = load_obj(Path(base_dir, 'peak_sampler_mz_rt_int_19_beers_fullscan.p'))
hmdb = load_obj(Path(base_dir, 'hmdb_compounds.p'))

Set ViMMS logging level

In [7]:
set_log_level_warning()
# set_log_level_info()
# set_log_level_debug()

## Create Initial Chemical

Define an output folder containing our results

In [8]:
out_dir = Path(base_dir, 'results', 'MS1_multiple')

Here we generate multiple chemical objects that will be used across samples. The chemical objects are generated by sampling from metabolites in the HMDB database.

In [9]:
# the list of ROI sources created in the previous notebook '01. Download Data.ipynb'
ROI_Sources = [str(Path(base_dir,'DsDA', 'DsDA_Beer', 'beer_t10_simulator_files'))]

# minimum MS1 intensity of chemicals
min_ms1_intensity = 1.75E5

# m/z and RT range of chemicals
rt_range = [(400, 800)]
mz_range = [(100, 400)]

# the number of chemicals in the sample
n_chems = 1000

# maximum MS level (we do not generate fragmentation peaks when this value is 1)
ms_level = 1

# for this experiment, we restrict the sampled chromatograms to be within 20 - 40s in length
# so they are not too big and too small
roi_rt_range = [20, 40]

In [10]:
chems = ChemicalCreator(ps, ROI_Sources, hmdb)
dataset = chems.sample(mz_range, rt_range, min_ms1_intensity, n_chems, ms_level, roi_rt_range=roi_rt_range)
save_obj(dataset, Path(out_dir, 'BaseDataset', 'dataset.p'))

Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\BaseDataset\dataset.p


In [11]:
for chem in dataset[0:10]:
    print(chem)

KnownChemical - 'C21H28O2' rt=732.88 max_intensity=268684.17
KnownChemical - 'C9H15NO3S' rt=626.68 max_intensity=2101106.53
KnownChemical - 'C12H22O4' rt=548.52 max_intensity=743724.75
KnownChemical - 'C16H38N2' rt=577.29 max_intensity=1230168.35
KnownChemical - 'C5H9NO2' rt=382.06 max_intensity=8778690.16
KnownChemical - 'C9H15N5O' rt=429.14 max_intensity=402739.66
KnownChemical - 'C15H21N3O' rt=526.60 max_intensity=8337695.01
KnownChemical - 'C10H15N3O2' rt=422.79 max_intensity=311677.09
KnownChemical - 'C12H14N4O2S' rt=658.45 max_intensity=65078397.89
KnownChemical - 'C6H14O6S2' rt=428.27 max_intensity=933174.98


## Create Multiple Samples

The next section allows us to define classes of biological replicates, each having multiple technical replicates. 

Below we create two biological classes ('class0', 'class1'), each having 10 technical replicates with some noise on the chemical's intensity.

In [12]:
n_samples = [10, 10] # number of files per class
classes = ["class%d" % i for i in range(len(n_samples))] # creates default list of classes
intensity_noise_sd = [1000] # noise on max intensity

In [13]:
classes

['class0', 'class1']

Add intensity changes between different classes

In [14]:
change_probabilities = [0 for i in range(len(n_samples))] # probability of intensity changes between different classes
change_differences_means = [0 for i in range(len(n_samples))] # mean of those intensity changes
change_differences_sds = [0 for i in range(len(n_samples))] # SD of those intensity changes

Add experimental variables (examples in comments)

In [15]:
experimental_classes = None # [["male","female"],["Positive","Negative","Unknown"]]
experimental_probabilitities = None # [[0.5,0.5],[0.33,0.33,0.34]]
experimental_sds = None # [[250],[250]]

Dropout chemicals in different classes

In [16]:
# drop-out chemicals by their probabilities
dropout_probability = 0.2
dropout_probabilities = [dropout_probability for i in range(len(n_samples))]
dropout_numbers = None # drop-out chemicals by an absolute number

# dropout_probabilities = None
# dropout_numbers = 2 # number of chemicals dropped out in each class

Generate multiple samples

In [17]:
save_location = os.path.join(out_dir, 'ChemicalFiles')

In [18]:
multiple_samples = MultiSampleCreator(dataset, n_samples, classes, intensity_noise_sd, 
                                      change_probabilities, change_differences_means, change_differences_sds, dropout_probabilities, dropout_numbers,
                                     experimental_classes, experimental_probabilitities, experimental_sds, save_location=save_location)

Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\ChemicalFiles\sample_0.p
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\ChemicalFiles\sample_1.p
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\ChemicalFiles\sample_2.p
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\ChemicalFiles\sample_3.p
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\ChemicalFiles\sample_4.p
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\ChemicalFiles\sample_5.p
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\ChemicalFiles\sample_6.p
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\ChemicalFiles\sample_7.p
Saving <class 'l

In [19]:
total_samples = np.sum(multiple_samples.n_samples)
total_samples

20

We can also print the chemicals that are missing (removed by drop-out) in each class.

In [20]:
save_obj(multiple_samples.missing_chemicals, Path(out_dir, 'MissingChemicals', 'missing_chemicals.p'))
multiple_samples.missing_chemicals

Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\MissingChemicals\missing_chemicals.p


[[KnownChemical - 'C21H28O2' rt=732.88 max_intensity=268684.17,
  KnownChemical - 'C12H22O4' rt=548.52 max_intensity=743724.75,
  KnownChemical - 'C9H15N5O' rt=429.14 max_intensity=402739.66,
  KnownChemical - 'C10H15N3O2' rt=422.79 max_intensity=311677.09,
  KnownChemical - 'C16H19NOS' rt=664.34 max_intensity=498105.07,
  KnownChemical - 'C10H15N3O5' rt=633.61 max_intensity=531359.64,
  KnownChemical - 'C6H13NO2' rt=371.39 max_intensity=3724193.90,
  KnownChemical - 'C6HCl5O' rt=414.97 max_intensity=410884.25,
  KnownChemical - 'C16H14ClN3O' rt=577.63 max_intensity=263971.98,
  KnownChemical - 'C17H21N' rt=456.76 max_intensity=588754.25,
  KnownChemical - 'C18H24O3' rt=736.21 max_intensity=209830.14,
  KnownChemical - 'C7H8O5S' rt=412.30 max_intensity=2309042.63,
  KnownChemical - 'C9H11N5O3' rt=668.43 max_intensity=5617453.25,
  KnownChemical - 'C15H15NO3' rt=682.22 max_intensity=394247.65,
  KnownChemical - 'C9H21N3O' rt=574.84 max_intensity=1280653.26,
  KnownChemical - 'C13H16N2O2

## Run MS1 controller on the samples and generate .mzML files

We can now take the multiple samples created above and generate mass spectral data (.mzML files) using the MS1 controller in ViMMS.

In [21]:
min_rt = rt_range[0][0]
max_rt = rt_range[0][1]
controllers = defaultdict(list)
controller_to_mzml = {}

mzml_dir = Path(out_dir, 'mzMLFiles')
num_classes = len(n_samples)
sample_idx = 0
for j in range(num_classes): # loop over classes
    num_samples = n_samples[j]
    for i in range(num_samples): # loop over samples for each class
        
        # load the sample
        fname = Path(save_location, 'sample_%d.p' % sample_idx) 
        sample = load_obj(fname)
        sample_idx += 1
        
        # define output .mzML filename
        out_file = 'number_%d_class_%d.mzML' % (i, j)
        out_path = Path(mzml_dir, out_file)
        print('Generating %s' % out_path)

        # run it through the MS1 controller        
        mass_spec = IndependentMassSpectrometer(POSITIVE, sample, ps)
        controller = SimpleMs1Controller(mass_spec)
        controller.run(min_rt, max_rt)
        controller.write_mzML('my_analysis', out_path)

        # save the resulting controller
        controllers[j].append(controller)
        controller_to_mzml[controller] = (j, out_file, )

Generating C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\mzMLFiles\number_0_class_0.mzML


400.81993000000045it [00:04, 92.48it/s]                                                                              


Created C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\mzMLFiles
Generating C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\mzMLFiles\number_1_class_0.mzML


400.7331000000005it [00:04, 95.30it/s]                                                                               


Generating C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\mzMLFiles\number_2_class_0.mzML


...

400.75644999999827it [00:04, 88.37it/s]                                                                              


Generating C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\mzMLFiles\number_8_class_1.mzML


400.00880000000143it [00:04, 90.58it/s]                                                                              


Generating C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\mzMLFiles\number_9_class_1.mzML


400.82949000000076it [00:04, 80.46it/s]                                                                              


## Print out the missing peaks

The controller object contains all the information about the state of the mass spectrometry process over time. Below we demonstrate this by generating a report of peaks corresponding to a chemical that are present in one class but is missing from the other class. This can be useful in the benchmark evaluation of peak picking or alignment algorithms.

In [22]:
def get_chem_to_peaks(controller):
    chem_to_peaks = defaultdict(list)
    frag_events = controller.mass_spec.fragmentation_events
    for frag_event in frag_events:
        chem = frag_event.chem
        peaks = frag_event.peaks
        chem_to_peaks[chem].extend(peaks)
    return chem_to_peaks

In [23]:
for controller, (current_class, mzml_filename) in controller_to_mzml.items():
    controller_peaks = get_chem_to_peaks(controller)
    basename = os.path.basename(mzml_filename)
    front, back = os.path.splitext(mzml_filename)
    outfile = front + '.csv'

    missing_peaks = []            
    for other_class in range(num_classes):
        if current_class == other_class:
            continue

        # get the peaks that are present in current_class but missing in other_class
        missing_chems = multiple_samples.missing_chemicals[other_class]
        for chem in missing_chems:
            peaks = controller_peaks[chem]
            for peak in peaks:
                row = (chem.formula.formula_string, current_class, other_class, peak.mz, peak.rt, peak.intensity)
                missing_peaks.append(row)
    
    # convert to dataframe
    columns = ['formula', 'present_in', 'missing_in', 'mz', 'RT', 'intensity']
    missing_df = pd.DataFrame(missing_peaks, columns=columns)
    missing_df.to_csv(os.path.join(out_dir, 'MissingChemicals', os.path.basename(outfile)))

In [24]:
missing_df.head(20)

,formula,present_in,missing_in,mz,RT,intensity
0,C21H28O2,1,0,313.216480,733.48489,190887.063824
1,C21H28O2,1,0,330.243026,733.48489,36691.272425
2,C21H28O2,1,0,389.128243,733.48489,89999.078686
3,C21H28O2,1,0,642.451956,733.48489,52228.312111
4,C21H28O2,1,0,314.219835,733.48489,44585.350600
5,C21H28O2,1,0,315.223189,733.48489,4958.936872
6,C21H28O2,1,0,313.216438,734.96489,201067.826579
7,C21H28O2,1,0,330.242984,734.96489,38648.163229
8,C21H28O2,1,0,389.128201,734.96489,94799.085820
9,C21H28O2,1,0,642.451914,734.96489,55013.854746
